# Using Postgres to Select Features

In [ ]:
PG_HOST = 'localhost'
PG_PORT = 32780
DATA_FILE = 'data/big_author_data.p'

The objective is to predict those that tweet again. To achieve this:

 * Take the first tweet of every unique author
 * Split them into those that tweet again and those that don't
 * Split them again into 3 groups

In [ ]:
import pickle

with open(DATA_FILE, 'rb') as handle:
    author_data = pickle.load(handle)

In [ ]:
from collections import defaultdict

docs_by_author = defaultdict(list)

for doc in author_data:
    docs_by_author[doc['author']].append(doc)

for docs in docs_by_author.values():
    docs.sort(key=lambda doc: doc['date'])

In [ ]:
repeat_authors = {
    author: docs
    for author, docs in docs_by_author.items()
    if len(docs) > 1
}
one_shot_authors = {
    author: docs[0]
    for author, docs in docs_by_author.items()
    if len(docs) == 1
}

In [ ]:
print(f'Repeat Authors: {len(repeat_authors)}')
print(f'One Shot Authors: {len(one_shot_authors)}')
print(f'{100 * len(retweeters) / len(author_data)}% Repeat Authors')

In [ ]:
def to_sets(repeaters, one_shotters):
    return [
        {
            'repeat authors': r,
            'one shot authors': o
        }
        for r, o in zip(to_threes(repeaters), to_threes(one_shotters))
    ]

def to_threes(data):
    data_items = list(data.items())
    step = len(data) // 3
    return [
        to_dict(collection)
        for collection in [
            data_items[:step], data_items[step:step * 2], data_items[step * 2:]
        ]
    ]

def to_dict(items):
    return {key: value for key, value in items}

investigation, testing, validation = to_sets(repeat_authors, one_shot_authors)

In [ ]:
len(investigation['repeat authors']) + len(testing['repeat authors']) + len(validation['repeat authors'])